# Slope Analysis

This project use the change of fluorecent intensity slope to identify responders from calcium imaging experiment.

## Analysis Steps

The `getBaselineAndMaxDrugSlope` function smoothes the raw data by the moving window decided by `filterSize`, and analyzes the smoothed Ca intensity in an CSV and returns baseline slope and drug slope.

The _slope of baseline_ is calculated as the linear regreasion slope during the 3 minutes period before stimulation onset.

In addition, the smoothed data are separated into segments which n = regressionSize data points are included. The linear regression slope is then calculated for each segment. 

The _peak slope of stimulation_ is the most negative slope during the chosen stimulation period .

## Set-Up the Environment

In [1]:
%load_ext autoreload
import sys
sys.path.append("../src")
from os.path import basename
import slopeTools
import plotTools
import statsTools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Define CSV Files and Filter Settings

The user can list the ABF files they want to analyze

In [2]:
def getBaselineAndMaxStimulationSlopeFromCSV(rawTimes,rawIntensity, filterSize = 15, regressionSize = 15):
    """
    This method analyzes fluorecent intensuty in a CSV and returns baseline slope and stimulation slope.
    
    Arguments:
        filterSize: number of points (sweeps) for the moving window average
        regressionSize: number of points (sweeps) to use to calculate regression slopes during the stimulation range
        
    Returns:
        baseline regression slope (over full range)
        peak stimulus regression slope (regression over defined size)
    """

    plt.plot(rawTimes, rawIntensity, '.', alpha=.5)
    smoothIntensity, smoothTimes = statsTools.smoothY(rawIntensity, rawTimes, filterSize)
    plt.plot(smoothTimes, smoothIntensity, '-')

    # determine the time range of stimulus
    stimulationTimeStart = rawTimes[30]
    stimulationSearchWidth = 30 # index
    stimulationTimeEnd = rawTimes[30 + stimulationSearchWidth]
    plt.axvspan(stimulationTimeStart, stimulationTimeEnd, color='r', alpha=.1)

    # determine baseline region based on stimulation time
    baselineTimeStart = 0
    baselineTimeEnd = stimulationTimeStart
    baselineIndexStart, baselineIndexEnd = statsTools.rangeIndex(smoothTimes, baselineTimeStart, baselineTimeEnd)
    baselineIntensity = smoothIntensity[baselineIndexStart:baselineIndexEnd]
    plt.axvspan(baselineTimeStart, baselineTimeEnd, color='b', alpha=.1)

    # isolate smoothed baseline intensity
    baselineIntensity = smoothIntensity[baselineIndexStart:baselineIndexEnd]
    baselineTimes = smoothTimes[baselineIndexStart:baselineIndexEnd]
    baselineSlope, baselineIntercept, r, p, stdErr = scipy.stats.linregress(baselineTimes, baselineIntensity)

    # calculate linear regression of baseline region
    baselineRegressionXs = np.linspace(baselineTimeStart, baselineTimeEnd)
    baselineRegressionYs = baselineRegressionXs * baselineSlope + baselineIntercept
    plt.plot(baselineRegressionXs, baselineRegressionYs, color='b', ls='--')
    print(f"Baseline slope: {baselineSlope} pA/min")

    # perform a moving window linear regression on the smoothed Intensity
    segments = statsTools.getMovingWindowSegments(smoothIntensity, regressionSize)
    segSlopes = getAllSegmentSlopes(segments, experimentPeriod)   
    segTimesOffset = (regressionSize * experimentPeriod)
    segTimes = np.arange(len(segSlopes)) * experimentPeriod + segTimesOffset    
    plt.subplot(212, sharex = ax1)
    plt.plot(segTimes, segSlopes, '.')

    # search the stimulation range for the most negative slope
    plt.axvspan(stimulationTimeStart, stimulationTimeEnd, color='r', alpha=.1)
    stimulationSlopeMin = statsTools.rangeMin(segSlopes, segTimes, stimulationTimeStart, stimulationTimeEnd)
    stimulationSlopeMinIndex = segSlopes.index(stimulationSlopeMin)
    stimulationSlopeMinTime = segTimes[stimulationSlopeMinIndex]
    print(f"stimulation slope: {stimulationSlopeMin} %/min")
    plt.axvline(stimulationSlopeMinTime, color='r', ls='--')
    plt.axhline(stimulationSlopeMin, color='r', ls='--')
    plt.axhline(baselineSlope, color='b', ls='--')

    plt.ylabel("Slope (pA/min)")
    plt.xlabel("Time (minutes)")

    plt.show()
    
    return baselineSlope, stimulationSlopeMin

In [3]:
csvFilePaths = [R"X:\Data\OT-Cre\OT-GCaMP-nonspecific\04-03-19 evoke OT\04-30-2020 K-GLU analyze\025um.xls",
                R"X:\Data\OT-Cre\OT-GCaMP-nonspecific\04-03-19 evoke OT\04-30-2020 K-GLU analyze\050um.xls"
               ]

The users can decide the parameters they want for data analysis. 

`filterSize` decides number of points (sweeps) for the moving window average. 

`regressionSize` decides the number of smoothed data points used to calculate linear regression slopes during the stimulation range.

In [4]:
filterSize = 15
regressionSize = 15

## Analyze All ABFs

In [7]:
baselineSlopes = []
stimulationSlopes = []
for csvFilePath in csvFilePaths:
    csvData = np.loadtxt(csvFilePaths, delimiter=",")
    header = csvData[0,:]
    rawTimes = np.arange(csvData[1:,0]) *5/60 #minutes 
    print(csvData)
    experimentPeriod = len(csvData[1:,0]) *5/ 60.0 # minutes
    for i in range(len(header)):
        plt.figure(figsize=(8, 6))
        ax1 = plt.subplot(211)
        plt.title(header[i])
        plt.ylabel("dF/F(%)")
        rawIntensity = csvData[:,i+1]
        baselineSlope, stimulationSlope = getBaselineAndMaxStimulationSlopeFromCSV(abfFilePath, filterSize, regressionSize)
        baselineSlopes.append(baselineSlope)
        stimulationSlopes.append(stimulationSlope)

ValueError: could not convert string to float: 'X:\\Data\\OT-Cre\\OT-GCaMP-nonspecific\\04-03-19 evoke OT\\04-30-2020 K-GLU analyze\\025um.xls'

## Compare Baseline vs. Stimulation Slopes

The users can plot the basleine slope and the peak stimulation slope of each cell, and report the p-value in the title by performing a paired t-test between baseline slopes and peak stimulation slopes.

In [ ]:
plotTools.plotPairs(baselineSlopes, stimulationSlopes, "slopes")

## Assess Responsiveness of All Cells

Generate a scatter plot showing the slope difference of each cell.

In [ ]:
slopeThreshold = -1.5 
stimulationEffects = []
for i in range(len(abfIDs)):
    stimulationEffects.append(stimulationSlopes[i] - baselineSlopes[i])

plt.figure (figsize=(6, 4))
plt.ylabel("Slope Difference (%/min)")
plt.plot(abfIDs, stimulationEffects, 'o', color = "b")
plt.gca().set_xticklabels(abfIDs, rotation=45, ha='right')
plt.axhline(slopeThreshold, color='r', ls='--')
plt.show()

## Define Cells as Responsive vs. Non-Responsive

The users can define the <b>slopeThreshold</b>. The difference between baseline slope and peak stimulation slope must be more negative than this value to be a responder
slopeThreshold

In [ ]:
stimulationEffects=statsTools.responderByDelta(abfIDs, stimulationEffects, slopeThreshold)